In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [38]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()

    except AttributeError:

#         try:
#             # If there is some deal price
#             price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()

#         except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count



In [39]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))


    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)


In [40]:
amazon_df

,title,price,rating,reviews
3,Wesley Milestone 2.0 15.6 inch 30 L Casual Wat...,$76.00,4.3 out of 5 stars,"5,617 ratings"
4,Lavie Sport Lino Large Size 63 cms Wheel Duffe...,$98.00,3.9 out of 5 stars,"2,724 ratings"
5,ADISA 15.6 inch Laptop Backpack Office Bag Col...,Page 1 of 1,3.8 out of 5 stars,551 ratings
7,Half Moon 35 L Waterproof Laptop College Schoo...,Page 1 of 1,4.7 out of 5 stars,"14,750 ratings"
8,SAFARI 15 Ltrs Sea Blue Casual Backpack (DAYPA...,Page 1 of 1,4.0 out of 5 stars,"17,650 ratings"
9,Gear Adult Casual Backpack One Size Black And ...,Page 1 of 1,3.7 out of 5 stars,"5,966 ratings"
10,"DELL PO1520PS Laptop Bag 15"" Backpack Black Gr...",$61.50,4.6 out of 5 stars,"1,590 ratings"
12,Half Moon Strike 35L Unisex Spacious School/ L...,Page 1 of 1,Previous page of related Sponsored Products,
13,Kuber Industries Handcrafted Embroidered Clutc...,Page 1 of 1,3.8 out of 5 stars,471 ratings
17,Blue Tree Boy's and Girl's Plush Soft Spiderma...,Page 1 of 1,4.0 out of 5 stars,"1,030 ratings"
